hello evryone,
This notebook goes through the whole process of model training from EDA , data wrngling to feature engineering to processingg the ct data to final model training 

please upvote if you like my work

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
%matplotlib inline

warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test_df = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub_df = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')

# Data Exploration and Feature Engineering
----
----

In [ ]:
train_df.head()

In [ ]:
print(f'number of unique patients = {len(train_df.patient_id.unique())}')

In [ ]:
plt.figure(figsize=(4, 6))
sns.countplot(x = 'benign_malignant' , data=train_df)
plt.title('class distribution')

okay so we have a class imbalance problem

In [ ]:
print(train_df.shape)
train_df.isnull().sum()

In [ ]:
plt.figure()
sns.distplot(train_df.age_approx)
plt.title('Distribution of Age')

As we can see that the Age has discrete values and a normal distribtuon. we can use countplot to better visualize the age feature and how our outcome is dependent on AGE

In [ ]:
train_df.age_approx.value_counts()

In [ ]:
fig = plt.figure(figsize=(10, 16))
fig.add_subplot(2,1,1)
sns.countplot(x='age_approx' , data = train_df)
plt.title('freq count of AGE')

fig.add_subplot(2,1,2)
sns.countplot(x='benign_malignant' , hue='age_approx' , data=train_df)
plt.title('distribution of age with respect to outcome')

disttribution by sex

In [ ]:
sns.countplot(x='sex' , data= train_df)
plt.title('distribution by sex')

now lets explore anatom_site_general_challenge feature

In [ ]:
train_df.anatom_site_general_challenge.value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='anatom_site_general_challenge' , data=train_df)
plt.title('dustribution of anatom_site_general_challenge')

now lets handle the missing values

* ## 1. AGE - we can fill the missing values with mean

In [ ]:
train_df['age_approx'] = train_df.age_approx.fillna(np.mean(train_df['age_approx']))
test_df['age_approx'] = test_df.age_approx.fillna(np.mean(test_df['age_approx']))

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

* ## 2. SEX -- 
if the unknown values are also present in the test set we will replace the nan with an unknown class U otherwise we will look to find its value in train set first if not found we will drop the unknown

In [ ]:
unknown_sex= train_df.loc[train_df.sex.isnull()]
un_sex_list = unknown_sex.patient_id.unique()

In [ ]:

for p in un_sex_list:
    a = train_df.loc[(train_df.patient_id == p) & (train_df.sex.notnull()) , 'sex']
    print(a)
    #train_df.loc[(train_df.patient_id == p)].fillna(a , inplace=True)

so there are only two pateints whose sex we don't know so now lest look if there are any null values for sex in test set or not

In [ ]:
test_df.isnull().sum()

now since there are no null values for sex in the test set we can drop these two pateints out of our training set

In [ ]:
train_df.dropna(subset=['sex'] , inplace=True)
print(train_df.shape)
train_df.isnull().sum()

converting sex to a numerical feature

In [ ]:
train_df['sex'] = train_df['sex'].replace('male' , 0)
train_df['sex'] = train_df['sex'].replace('female' , 1)

test_df['sex'] = test_df['sex'].replace('male' ,0)
test_df['sex'] = test_df['sex'].replace('female' , 1)

train_df.dtypes

* ## 3. anatom_site_general_challenge --
now we numerically encode the labels and use unknown class U for unknown labels

In [ ]:
# filling nan with U
train_df['anatom_site_general_challenge'] = train_df['anatom_site_general_challenge'].fillna('U')
test_df['anatom_site_general_challenge'] = test_df['anatom_site_general_challenge'].fillna('U')
print(len(train_df['anatom_site_general_challenge'].unique()))

In [ ]:
num_enc_dict = { p: i for i, p in enumerate(train_df['anatom_site_general_challenge'].unique())}
num_enc_dict

In [ ]:
train_df['anatom_site_general_challenge'] = train_df['anatom_site_general_challenge'].map(num_enc_dict)
test_df['anatom_site_general_challenge'] = test_df['anatom_site_general_challenge'].map(num_enc_dict)

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

* at last fixing the diagnosis columns

In [ ]:
num_enc_dict = { p: i for i, p in enumerate(train_df['diagnosis'].unique())}
num_enc_dict

In [ ]:
train_df['diagnosis'] = train_df['diagnosis'].map(num_enc_dict)
#test_df['diagnosis'] = test_df['diagnosis'].map(num_enc_dict)

Hurrah! we replaced all the unknown values with a proper value and now both our test and train set are clean and contain no null values

now for once final time lets look at the count of all the features in the dataset

In [ ]:
fig = plt.figure(figsize=(12,10))
i =1
for p in train_df.columns[2:]:
    ax = fig.add_subplot(3,3,i)
    sns.countplot(x=p, data=train_df)
    ax.set_xticks([])
    i +=1

fig.tight_layout(pad=3)

# Image data processing
now lets look at image data

In [ ]:
import pydicom
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import io
import gc

In [ ]:
train_path = os.path.join('../input/siim-isic-melanoma-classification/jpeg/train/')
test_path = os.path.join('../input/siim-isic-melanoma-classification/jpeg/test/')

In [ ]:
TFRpath = os.path.join('../input/siim-isic-melanoma-classification/tfrecords/')

In [ ]:
for f in os.listdir(train_path)[:30]:
    a = cv2.imread(train_path + f)
    print(a.shape)

so we can see that the images are pretty big and we can use fed these directly to the the convnets so we will need to reduce the size of of these images

In [ ]:
# image dimension
IMG_DIM = 64
IMG_DEPTH = 3
BATCH_SIZE = 32


In [ ]:
raw_data = tf.data.TFRecordDataset('../input/siim-isic-melanoma-classification/tfrecords/train00-2071.tfrec')
for r in raw_data.take(1):
    ex = tf.train.Example()
    ex.ParseFromString(r.numpy())
    print(ex)

In [ ]:
feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'image_name': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'target': tf.io.FixedLenFeature([], tf.int64, default_value=0),
}
def parse_data_func(ex_proto):
    return tf.io.parse_single_example(ex_proto , feature_description)

In [ ]:
dic = {}
parsed_data = raw_data.map(parse_data_func)
for p in parsed_data.take(10):
    img = p['image'].numpy()
    name = p['image_name'].numpy()
    name = name.decode('utf-8')
    img = np.array(Image.open(io.BytesIO(img)))
    img = cv2.resize(img , (IMG_DIM , IMG_DIM))
    print(img.shape)
    di = {name : img}
    dic.update(di)

now we can use the same approcah to write our function

In [ ]:
parsed_data = raw_data.map(parse_data_func)

In [ ]:
# function to load TFR data into a dictionary
def load_TFR_data(sample, img_dim):
    img = sample['image'].numpy()
    name = sample['image_name'].numpy()
    name = name.decode('utf-8')
    img = np.array(Image.open(io.BytesIO(img)))
    img = cv2.resize(img , (img_dim , img_dim))
    img = img/255
    img = img.astype(np.uint8)
    return {name : img}

In [ ]:
train_ct_data = {}
for p in parsed_data:
    train_ct_data.update(load_TFR_data(p , IMG_DIM))

len(train_ct_data)

In [ ]:
def load_ct_data_to_dict(directory , _set='train'):
    data_dict = {}
    for p in os.listdir(directory):
        if p.startswith(_set):
            path = directory+p
            raw_data = tf.data.TFRecordDataset(path)
            parsed_data = raw_data.map(parse_data_func)
            for p in parsed_data:
                data_dict.update(load_TFR_data(p , IMG_DIM))
    return data_dict
            
                

In [ ]:
train_ct_data_dict = load_ct_data_to_dict(TFRpath , _set='train')
test_ct_data_dict = load_ct_data_to_dict(TFRpath , _set='test')

print(f'number of samples of ct in train set = {len(train_ct_data_dict)}')
print(f'number of samples of ct in test set = {len(test_ct_data_dict)}')

In [ ]:
x_train_images = []
for p in train_df['image_name']:
    x_train_images.append(train_ct_data_dict[p])
    del(train_ct_data_dict[p])
    
x_train_images = np.array(x_train_images)
    

In [ ]:
x_train_images.shape

In [ ]:
x_test_images = []
for p in test_df['image_name']:
    x_test_images.append(test_ct_data_dict[p])
    del(test_ct_data_dict[p])
    
x_test_images = np.array(x_test_images)
x_test_images.shape

In [ ]:
np.save('x_train_images.npy' , x_train_images)
np.save('x_test_images.npy' , x_test_images)

In [ ]:
gc.collect()

In [ ]:
x_train_images.nbytes

now we have  training and testing images preparesd
lets prepare our numerical data


In [ ]:
train_df.columns

In [ ]:
x_train_numerical = train_df[['sex' ,'age_approx' ,'anatom_site_general_challenge']]
x_train_numerical = np.array(x_train_numerical)
y_train = train_df[['target']]
y_train = np.array(y_train)
x_test_numerical = test_df[['sex' ,'age_approx' ,'anatom_site_general_challenge']]
x_test_numerical = np.array(x_test_numerical)

print(f'shape of train images : {x_train_images.shape}')
print(f'shape of train numerical : {x_train_numerical.shape}')
print(f'shape of train target (y) : {y_train.shape}')
print(f'shape of test images : {x_test_images.shape}')
print(f'shape of test numerical : {x_test_numerical.shape}')



now our data is ready for training and we can go on to create our model

# model creation and training

we will be using a two branch model which will process the different tupes inputs
* branch 1 -- processes the image data
* branch 2 -- processes the tabular data
* final model will predict on [branch 1+ branch 2]
as shown in the figure 

![](https://www.pyimagesearch.com/wp-content/uploads/2019/02/keras_multi_input_design.png)


In [ ]:
from tensorflow.keras.layers import Conv2D , Activation , MaxPool2D , concatenate , GlobalAvgPool2D
from tensorflow.keras.layers import Flatten , Dense , Input , BatchNormalization , Dropout 
from tensorflow.keras.models import Model , Sequential

lets create the multilayer perceptron first

In [ ]:
def perceptron_model(dim):
    model = Sequential()
    model.add(Dense(12 , input_dim=dim , activation='relu'))
    model.add(Dense(8 , activation = 'relu'))
    return model

now the CNN

In [ ]:
def CNN_model(h,w,d):
    input_shape = (h,w,d)
    _input = Input(shape=input_shape)
    x = Conv2D(16 ,(3,3) , activation='relu' , padding='same')(_input)
    x = Conv2D(32 ,(3,3) , activation ='relu' , padding='same')(x)
    x = MaxPool2D((2,2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(64 , (3,3) , activation='relu' , padding='same')(x)
    x = MaxPool2D((2,2))(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(1024 , activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128 , activation='relu')(x)
    model = Model(inputs=_input , outputs=x)
    return model

now lets create a checkpoint so that we can store the weights of our model pause and continue the training the way we want. first lets create a directory to hold our checkpoint

In [ ]:
! mkdir model_checkpoints

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = 'model_checkpoints/model_weights.hdf5' , save_best_only = True , verbose =1)

now lets define our final model

In [ ]:
linear_model = perceptron_model(3)
Cnn_model = CNN_model(IMG_DIM , IMG_DIM, 3)

f = concatenate([linear_model.output , Cnn_model.output])
f = Dense(128 , activation = 'relu')(f)
f = Dropout(0.3)(f)
f = Dense(64 , activation = 'relu')(f)
f = Dropout(0.5)(f)
f = Dense(1 , activation = 'sigmoid')(f)

final_model = Model(inputs=[linear_model.input , Cnn_model.input] , outputs=f)

now our final model is ready and we can now compile it. remember the metric we need is roc_auc_score

In [ ]:
final_model.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'])
final_model.summary()

now our model is ready lets satrt the training

In [ ]:
history = final_model.fit(x=[x_train_numerical , x_train_images] , y = y_train , epochs=30 , batch_size=32 , callbacks=[cp_callback])

In [ ]:
preds = final_model.predict(x=[x_test_numerical , x_test_images])

In [ ]:
sub_df.head()

In [ ]:
sub_df['target'] = preds
sub_df.set_index('image_name' , inplace=True)
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv')

now this was a baseline model which didn't use any validation set. the score of this model was 0.5757 so now we create a validation and use early stopping to save our model from overfitting

In [ ]:
# creating validation set
# number of samples in validation set = k
k = 5000
x_val_numerical = x_train_numerical[len(x_train_numerical) - k :]
x_val_images = x_train_images[len(x_train_images) -k :]
y_val = y_train[len(y_train)-k:]

x_train_numerical = x_train_numerical[:len(x_train_numerical) - k]
x_train_images = x_train_images[:len(x_train_images) - k]
y_train = y_train[:len(y_train)-k]

print(f'shape of training numerical : {x_train_numerical.shape}')
print(f'shape of training images : {x_train_images.shape}')
print(f'shape of training target : {y_train.shape}')
print(f'shape of validation set (numerical) : {x_val_numerical.shape}')
print(f'shape of validation set (images) : {x_val_images.shape}')
print(f'shape of validation set (target) : {y_val.shape}')


and lets recompile our model which use auc score as a metrics

In [ ]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [ ]:
final_model.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=METRICS)

In [ ]:
# early_stopping
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc' , verbose=1, patience=10 , mode='max' ,restore_best_weights=True)

In [ ]:
history2 = final_model.fit(x= [x_train_numerical , x_train_images] , 
                           y = y_train , 
                           epochs = 50 , 
                           validation_data=([x_val_numerical , x_val_images] , y_val), 
                           callbacks = [cp_callback , es_callback])

In [ ]:
ax = plt.figure()
ax1 = plt.plot(history2.history['loss'])
ax2 = plt.plot(history2.history['val_loss'])
plt.title('LOSS')
plt.legend(labels =['loss' ,'val_loss'])
plt.show()

In [ ]:
plt.plot(history2.history['auc'])
plt.plot(history2.history['val_auc'])
plt.title('AUC')
plt.legend(labels = ['auc' ,'val_auc'])
plt.show()

now lets make a prediction with this model

In [ ]:
preds = final_model.predict([x_test_numerical , x_test_images])

In [ ]:
sub_df['target'] = preds
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv')

so this time our score improved to 0.6770

now one thing we can also do is to get rid of class imabalnce as we saw in the beginning there is a huge imbalnce between classes so we need to handle this imbalnce

the techninque we are going to use is class weighting

In [ ]:
# first lets look at the count of each class in our training sample
pos = np.count_nonzero(y_train==0)
neg = np.count_nonzero(y_train==1)
total = len(y_train)
print('class 0 : ' + str(pos))
print('class 1 : ' + str(neg))
print('total : ' +str(total))

this is a huge imbalnce. so lets how weighting of classes is done

In [ ]:
w_0 = (1/pos)*(total)/2
w_1 = (1/neg)*(total)/2

class_weights = {0:w_0 ,1:w_1}
print(f'weight of class 0 : {w_0}')
print(f'weight of class 1 : {w_1}')

now we will feed this class weight to the model while training this will penalise the model weights for the imbalance between classes

In [ ]:
history3 = final_model.fit(x= [x_train_numerical , x_train_images] , 
                           y = y_train , 
                           epochs = 50 , 
                           validation_data=([x_val_numerical , x_val_images] , y_val), 
                           callbacks = [cp_callback , es_callback],
                           class_weight=class_weights)

In [ ]:
ax = plt.figure()
ax1 = plt.plot(history3.history['loss'])
ax2 = plt.plot(history3.history['val_loss'])
plt.title('LOSS')
plt.legend(labels =['loss' ,'val_loss'])
plt.show()

In [ ]:
plt.plot(history3.history['auc'])
plt.plot(history3.history['val_auc'])
plt.title('AUC')
plt.legend(labels = ['auc' ,'val_auc'])
plt.show()

In [ ]:
preds = final_model.predict([x_test_numerical , x_test_images])

In [ ]:
sub_df['target'] = preds
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv')

we can see our val loss was going during the training so lets apply early stopping on val_loss and check how the model performs

In [ ]:
es_callback2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss' , verbose=1, patience=10 , mode='auto' ,restore_best_weights=True)

In [ ]:
history4 = final_model.fit(x= [x_train_numerical , x_train_images] , 
                           y = y_train , 
                           epochs = 50 , 
                           validation_data=([x_val_numerical , x_val_images] , y_val), 
                           callbacks = [cp_callback , es_callback2],
                           class_weight=class_weights)

In [ ]:
z = final_model.evaluate([x_val_numerical , x_val_images] , y_val)

In [ ]:
preds = final_model.predict([x_test_numerical , x_test_images])


In [ ]:
sub_df['target'] = preds
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv')